<a href="https://colab.research.google.com/github/ydydydydydy/Colab/blob/main/ex04_%EA%B0%9C_%EA%B3%A0%EC%96%91%EC%9D%B4_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CNN(Convolution Neural Network)
  - 이미지 학습 가능(2차원의 데이터도 학습)
  - 데이터에서 특징을 추출하고 추출된 특징을 기반으로 학습
    - 이미지의 크기, 방향 등에 크게 관여하지 않는다
    

##### 축소샘플링
      - Conv(특징찾기) : 이미지에 적용시키면 전체 데이터의 크기가 줄어듦
      - 축소 샘플링 진행할 때 :
          1. 데이터의 크기가 너무 클 때
          2. 가장자리의 데이터가 필요없을 때
           padding = 'valid'
      - 축소 샘플링 진행하지 않을 때 :
          1. 데이터가 너무 작을 때
          2. 가장자리에 중요한 데이터가 있을 때
            padding = 'same'

Pooling
  - 특정 지역의 대표값만 추출
  - 축소샘플링이 이루어짐 > 특징 집약적인 데이터가 만들어짐 > 학습시간은 감소 > 성능은 비슷

Flatten
  - 다차원의 데이터를 1차원으로 만들어주는 역할
  - Conv > Pool > Flatten > Dense

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
 # /content/drive/MyDrive/Colab Notebooks/DeepLearning/data/dogs_vs_cats_small
 # 데이터 경로지정
 train_dir = '/content/drive/MyDrive/Colab Notebooks/DeepLearning/data/dogs_vs_cats_small/train'
 valid_dir = '/content/drive/MyDrive/Colab Notebooks/DeepLearning/data/dogs_vs_cats_small/validation'

In [54]:
# 하나의 변수에 이미지 파일 전부다 합치기
# 픽셀값( 0 ~ 255 / 정수 ) (변경) --> ( 0 ~ 1 까지의 / 실수값 )
## 1. 숫자 크기 줄이기 > 연산량 감소
## 2. 분산(값이 분포해있는 범) 줄이기 > 연산의 오류 줄어듦
# 이미지 크기 맞춰주기 (150,150)
# 라벨링
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [55]:
# 픽셀값 변경 기능 만들기
# 묵시적 형변환 하기
## 프로그래밍에서 연산상에서 타입이 변경되도록 하기
generator = ImageDataGenerator(rescale = 1./255) # 1. = 1.0

In [56]:
# 하나의 변수에 이밎 파일 전부다 합치기
# 이미지 크기 동일하게 만들어주기
# 라벨링
train_generator = generator.flow_from_directory(
    directory = train_dir, # train 이미지 경로, 변환할 이미지
    target_size = (150,150),  # 변환할 이미지 크기
    batch_size = 100,  # 한번에 변환할 이미지 갯수
    class_mode = 'binary'  # 라벨링 방법, 다중분류 : categorical
)

Found 2000 images belonging to 2 classes.


In [57]:
valid_generator = generator.flow_from_directory(
    directory = valid_dir, # train 이미지 경로, 변환할 이미지
    target_size = (150,150),  # 변환할 이미지 크기
    batch_size = 100,  # 한번에 변환할 이미지 갯수
    class_mode = 'binary'  # 라벨링 방법, 다중분류 : categorical
)

Found 1000 images belonging to 2 classes.


In [58]:
# CNN 모델 설계
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [59]:
# Dense 만 사용 = MLP
# Conv, Pool이 주로 사용된다 = CNN
# 딥러닝 모델 생성
# 건물 토대 설계
model1 = Sequential()

# 입력층
model1.add(Conv2D(  # 특징 찾기
    filters = 32, # 찾을 특징의 갯수
    kernel_size = (3,3),  # 특징의 크기
    input_shape = (150, 150,3),  # 입력 데이터의 모양(3 = RGB) # 0 : 검은색, 255 : 흰색
    activation = 'relu',
    padding = 'same'
))

model1.add(MaxPool2D( # 특징이 아닌 부분 삭제하기
    pool_size = (2,2) # 기준 크기에서 1개의 특징만 가져오기 / 4개중에 한 개 사용하기 (가장 특징 값이 큰 것)
))


model1.add(Conv2D(  # 특징 찾기 (앞에서 찾은 특징에서 더 세부적인 부)
    filters = 32, # 찾을 특징의 갯수
    kernel_size = (3,3),  # 특징의 크기
    activation = 'relu'
))

model1.add(MaxPool2D( # 특징이 아닌 부분 삭제하기
    pool_size = (2,2) # 기준 크기에서 1개의 특징만 가져오기 / 4개중에 한 개 사용하기
))

################# 특징 추출부 끝 ####################
model1.add(Flatten())  # 특징추출부와 분류부를 이어주는 역할
################# 분류 분석 시작 ####################
model1.add(Dense(units = 32, activation = 'relu'))
model1.add(Dense(units = 64, activation = 'relu'))
model1.add(Dense(units = 128, activation = 'relu'))
model1.add(Dense(units = 64, activation = 'relu'))
model1.add(Dense(units = 32, activation = 'relu'))

# 출력층
model1.add(Dense(units = 1, activation = 'sigmoid'))

In [60]:
# 학습 방법 설정
model1.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
model1.fit(
    train_generator, # 학습 데이터(X_train, y_train이 합쳐져 있다)
    epochs = 20,
    validation_data = valid_generator
)

Epoch 1/20
20/20 [==============================] - 14s 471ms/step - loss: 0.6960 - accuracy: 0.5125 - val_loss: 0.6906 - val_accuracy: 0.5450


In [ ]:
import PIL.Image as pimg
import cv2

# 이미지 불러오기
pre_img = cv2.imread('/content/doonge.jpg',cv2.IMREAD_COLOR)

# 이미지 색상 변경하기
pre_img = cv2.cvtColor(pre_img, cv2.COLOR_BGR2RGB)

# 이미지 크기 변경하기
pre_img = cv2.resize(pre_img,(150,150))

# 이미지 차원 변경하기
pre_img = pre_img.reshape((1,150,150,3))

In [ ]:
# 0 = cat
# 1 = dog
# 대다수의 사람은 개라는 결과를 얻는다 = 모델이 학습이 덜 돼서 개의 규칙은 잘 찾았지만 고양이의 규칙은  찾지 못했기 때문
model1.predict(pre_img)

In [ ]:
# 부동소수점 : 소수점의 위치가 변하지 않는다
# 1.7243704e-35 = 1.7243704 * 10^(-35) / 0이라고 봐도 무방

In [ ]:
# train accuracy : 0.98
# val accuracy : 0.7
# > train은 학습이 잘 됐고, val은 학습이 안됐다 > 과대적합에 걸렸다
#  과대적합을 해소하는 방법
# 1. 데이터의 양을 늘린다(추가적으로 데이터를 수집) > 많이 어렵다
# 2. 모델을 더 깊게 구성한다 (층을 더 많이 쌓는다)
# 3. 데이터를 확장한다(가지고 있는 데이터를 다양하게 표현)

In [ ]:
# 데이터 확장을 하고 새롭게 학습한다
# 데이터를 확장하면 원본 데이터는 사라진다
# 모델 설계 > 확장 안된 이미지로 학습 > 결과 확인 > 만족스럽지 않다 > 데이터 확장 후 학습

In [ ]:
# 기존 모델에 확장 데이터로 학습하기
# 확장의 방법 골라주기
aug_generator = ImageDataGenerator(
    rescale = 1./255, # 픽셀값 조정
    rotation_range = 20, # 회전 범위
    width_shift_range = 0.1, # 수평 이동
    height_shift_range = 0.1, # 수직 이동
    shear_range = 0.1, # 반시계 방향 회전
    zoom_range = 0.1, # 확대/축소 비율
    horizontal_flip = True, # 수평 뒤집기
    fill_mode = 'nearest' # 가까운 값으로 비어있는 곳을 채운다
)

In [ ]:
train_aug_generator = aug_generator.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 100,
    class_mode = 'binary'
)

In [ ]:
model1.fit(
    train_aug_generator,
    epochs = 20,
    validation_data = valid_generator,
)

In [ ]:
# train은 낮아지고 valudation은 높아졌다